<a href="https://colab.research.google.com/github/livinNector/climate-change-hackathon/blob/main/climate_change_hackathon_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cond-rnn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl

In [ ]:
from cond_rnn import ConditionalRecurrent

In [ ]:
years = np.repeat(np.repeat(np.arange(2018,2023),12).reshape(1,-1),50,axis=0)
months = np.repeat(np.repeat(np.arange(12).reshape(1,12),5,axis=0).reshape(1,-1),50,axis=0)
years,months,years.shape,months.shape

In [ ]:
aqi_features = ["aqi","so2","nox","pm10","pm2.5","co","o3"]
meteorological_features = ["rainfall","max_humidity","min_humidity","temp_min","temp_max","wind_speed"]
temporal_features = ["month_sin","month_cos","year"]

time_series_features = aqi_features + meteorological_features + temporal_features
geo_spatial_features = ["lat","long","alt"]

features ={
    name:tf.random.normal((50,60))
    for name in aqi_features+meteorological_features
}
features["year"] = tf.constant(years)
features["months_sin"] = tf.constant(np.sin(months*2*np.pi/12))
features["months_cos"] = tf.constant(np.cos(months*2*np.pi/12))


for f in geo_spatial_features:
  features[f] = tf.random.normal((50,1))



dataset = tf.data.Dataset.from_tensor_slices(features)


In [ ]:
next(dataset.batch(12).as_numpy_iterator())

In [ ]:
next(dataset.window(12,stride=1,drop_remainder=True).flat_map(lambda ys:{k:value.batch(12) for k,value in ys.items()}).as_numpy_iterator()).shape

In [ ]:
# class TeacherForcer(tf.keras.layers.Layer):
#   def __init__(self,layer,output_length):
#     self.layer = layer
  
#   def call(self,inputs):
#     output
#     for i in range(1,output_length):



In [ ]:
inputs = [tf.keras.Input(shape=(60,1),name=name) for name in aqi_features+meteorological_features+temporal_features]

inputs_concat = tf.keras.layers.concatenate(inputs)
cond_inputs = [tf.keras.Input(shape=(1),name=name) for name in geo_spatial_features]
cond_concat = tf.keras.layers.concatenate(cond_inputs)

encoder_output = ConditionalRecurrent(tfl.LSTM(16,activation="relu"))([inputs_concat,cond_concat])

# x  = tfl.LSTM(12)(encoder_output)
x  = encoder_output
aqi_out = tfl.Dense(1,activation = "relu",name="aqi_out")(x)
temp_high_out = tfl.Dense(1,activation = "relu",name="temp_high_out")(x)
humid_high_out = tfl.Dense(1,activation = "relu",name="humid_high_out")(x)

encoder_model = tf.keras.Model(inputs = inputs+cond_inputs,outputs = encoder_output)
decoder_model = tf.keras.Model(inputs = encoder_output,outputs = [aqi_out,temp_high_out,humid_high_out])

model = tf.keras.Model(inputs = inputs+cond_inputs,outputs = [aqi_out,temp_high_out,humid_high_out])


In [ ]:
tf.keras.utils.plot_model(model,rankdir="LR",show_shapes=True)

```
data{
  lat : val,
  long : val,
  features: {
    f1:[vals],
    f2:[vals],
    f3:[vals],
  }
}
```

In [ ]:
def get_input_feature(name):
  
